<a href="https://colab.research.google.com/github/cche0214/HuggingFaceLLM/blob/main/02pipeline%E7%9A%84%E5%86%85%E9%83%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline

# pipeline集成了三个步骤，预处理，模型计算和后处理
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
from transformers import AutoTokenizer

# 检查点是针对于架构的权重，这里的架构就是DistilBERT
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# 这里的意思就是，找到这个检查点下的Tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]

# Tokenizer将原始的文本转换成tokenID（实际应该是原始文本->分词成token->token映射为tokenID
# return_tensors参数指定返回的tensor类型，可以是PyTorch、TensorFlow或者纯NumPy
# Transformers模型只接受张量输入
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [ ]:
# 输出包含两个键input_ids和attention_mask
# input_ids就是每个句子中token的ID
# 现在就是有了可以输入模型的数据，所以接下来下载模型
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [ ]:
from transformers import AutoModel

# 这里并没有加载模型头，也就是不能得到情感分类任务的结果
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
outputs = model(**inputs)

# 输入句子，输出称为hidden states隐状态，这个维度很大有三个维度，隐状态也被称为模型头（输入下游任务）
# [BatchSize, SequenceLength, Hiddensize]
# 分别是一次处理的序列数量，表示序列（句子）的长度和每个模型输入的向量维度
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [ ]:
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634],
         [-0.3454,  0.8824, -0.0426,  ..., -0.0993, -0.8329, -0.1065]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [ ]:
# 现在要导入一个带有序列分类头的模型，也就是情感分类模型
from transformers import AutoModelForSequenceClassification

# 这里的checkpoint跟单纯导入AutoModle一样，原因是两者的区别只是有没有下游的模型头
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)


In [ ]:
# 这里的维度小了很多，因为模型头已经接受了前面的高维向量作为输入，输出包含两个值（每种标签一个的）向量
print(outputs.logits.shape)

torch.Size([2, 2])


In [ ]:
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
# 预测出来的暂时还是对数几率，是模型最后一层输出的原始的、未标准化的分数
# 因此要知道最后的概率，需要经过softmax层，所有Transformers模型的输出都是logits
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [ ]:
import torch

# 经过softmax层之后才得到概率，现在需要查看模型看每个概率对应哪种标签
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [ ]:
# 现在知道第一个是负面的概率，第二个是积极的概率
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [1]:
# 测试更多的句子，分别比较自己的管道和transformers实现的管道
from transformers import pipeline

classifierTF = pipeline("sentiment-analysis")
classifierTF(
    [
        "I absolutely love this project—it exceeded every expectation I had.",
        "This is the most rewarding experience I’ve had in years.",
        "I’m extremely disappointed with the final outcome.",
        "The entire process was frustrating and poorly managed.",
        "The outcome is good, and it meets most of my expectations."
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998852014541626},
 {'label': 'POSITIVE', 'score': 0.9998852014541626},
 {'label': 'NEGATIVE', 'score': 0.9997631907463074},
 {'label': 'NEGATIVE', 'score': 0.9998045563697815},
 {'label': 'POSITIVE', 'score': 0.9998612403869629}]

In [7]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
inputs = [
    "I absolutely love this project—it exceeded every expectation I had.",
    "This is the most rewarding experience I’ve had in years.",
    "I’m extremely disappointed with the final outcome.",
    "The entire process was frustrating and poorly managed.",
    "The outcome is good, and it meets most of my expectations."
]

inputsid = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
print(inputsid)

{'input_ids': tensor([[  101,  1045,  7078,  2293,  2023,  2622,  1517,  2009, 14872,  2296,
         17626,  1045,  2018,  1012,   102,     0],
        [  101,  2023,  2003,  1996,  2087, 10377,  2075,  3325,  1045,  1521,
          2310,  2018,  1999,  2086,  1012,   102],
        [  101,  1045,  1521,  1049,  5186,  9364,  2007,  1996,  2345,  9560,
          1012,   102,     0,     0,     0,     0],
        [  101,  1996,  2972,  2832,  2001, 25198,  1998,  9996,  3266,  1012,
           102,     0,     0,     0,     0,     0],
        [  101,  1996,  9560,  2003,  2204,  1010,  1998,  2009,  6010,  2087,
          1997,  2026, 10908,  1012,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}


In [6]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [8]:
# 这里前面加上**是Python语法，表示把inputsid这个字典的每个key作为函数的输入
# outs = model(input_ids = inputsid["inputs_ids"], attention_mask = inputsid["attention_mask"])
outs = model(**inputsid)

# 模型最后一层输出，没有经过SoftMax层
print(outs)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.3467,  4.7259],
        [-4.3424,  4.7297],
        [ 4.6356, -3.7126],
        [ 4.7140, -3.8262],
        [-4.2596,  4.6234]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [9]:
import torch

predictions = torch.nn.functional.softmax(outs.logits, dim=-1)
print(predictions)
# 输出和直接用pipeline是一样的

tensor([[1.1475e-04, 9.9989e-01],
        [1.1481e-04, 9.9989e-01],
        [9.9976e-01, 2.3677e-04],
        [9.9980e-01, 1.9542e-04],
        [1.3871e-04, 9.9986e-01]], grad_fn=<SoftmaxBackward0>)


In [10]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [3]:
from transformers import BertConfig, BertModel

config = BertConfig()

# 使用默认配置创建模型会使用随机值对其进行初始化
# 模型可以运行得到结果，但是输出会胡言乱语
model = BertModel(config)

In [4]:
# config中包含许多用于构建模型的属性，是写死的，还没有训练
# 还没有训练的意思是，它的权重还是随机的，config展示的只是它结构的一些属性
# hidden_states隐状态向量的大小，num_hidden_layers定义了Transformers模型的参数
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.6",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [10]:
# 未训练的模型输出是胡言乱语，不过这个是基础模型我不知道怎么确认
outputs = model(**inputsid)
print(outputs)
print(outputs.last_hidden_state.shape)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-8.9886e-02, -1.2167e+00,  6.0527e-01,  ...,  5.2541e-02,
          -1.7462e-01, -1.1783e+00],
         [-2.8070e-01, -7.2294e-01, -1.0446e+00,  ..., -6.6824e-01,
           3.6832e-01, -3.7543e-01],
         [ 3.9131e-01, -2.0747e-01,  1.2605e+00,  ..., -9.4767e-01,
           7.6124e-01, -5.5230e-01],
         ...,
         [-8.9965e-02, -7.9332e-01, -7.6795e-02,  ...,  5.4175e-01,
           1.2704e+00,  9.4850e-01],
         [ 5.5318e-01,  1.8107e-01,  8.1903e-01,  ..., -8.3638e-03,
           3.7854e-02,  2.5596e-01],
         [ 7.1611e-02, -1.0711e+00,  9.3156e-01,  ...,  5.9356e-01,
           1.5741e+00,  5.5809e-01]],

        [[-3.4121e-01,  5.9108e-02,  2.2050e-01,  ...,  1.0801e+00,
           6.7322e-02, -8.4861e-01],
         [ 2.5620e-01, -7.0649e-01,  5.7098e-02,  ..., -8.2450e-01,
           6.4584e-01,  2.5623e-01],
         [ 2.2014e-01, -9.4383e-01,  1.2086e+00,  ..., -6.6348e-01,
           1.

In [12]:
# 加载已经训练过的模型，参考上一节的方法
from transformers import BertModel

# 这里和AutoModel的区别就是，你已经指定了Bert模型的架构，所以你的checkpoint必须是Bert模型的
# 所以这里的逻辑就是找到Bert模型的bert-base-cased检查点的模型
# 这个模型就是预训练后的模型了，可以执行任务，也可以在新任务上微调
# 因此从单个Model变成AutoModel，checkpoint就不受限制了，Transformers库会自动识别
model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [13]:
model.save_pretrained("directory_on_my_computer")

In [20]:
ls directory_on_my_computer
# config.json表示模型的结构和元数据
# model.safetensors表示模型的所有参数

config.json  model.safetensors


In [24]:
# 从自己的电脑下载模型
modelload = BertModel.from_pretrained("directory_on_my_computer")

In [26]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


In [28]:
# 这里就是指定BERT模型的分词器
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [31]:
# 同样的，你可以直接使用AutoTokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [32]:
tokenizer("Using a Transformers network is simple")
# 这个输出和情感分类的那个模型输出好像不同，多了一个token_type_ids

{'input_ids': [101, 7993, 170, 25267, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [34]:
tokenizer("Using a Transformers network is simple")
# 情感分类的输出

{'input_ids': [101, 2478, 1037, 19081, 2897, 2003, 3722, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [35]:
# 保存tokenizer
tokenizer.save_pretrained("directory_on_my_computer")

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json',
 'directory_on_my_computer/tokenizer.json')

In [49]:
# 详细展示Tokenizer内部的处理过程:原始文本->分词（不同模型的分词法不同）->映射词汇ID（词汇表)->转换成模型可以接受的张量
from transformers import AutoTokenizer

# 实例化一个BERT模型的分词器，你正常使用直接用就行，下面展示详细内部步骤
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformers network is simple, I like tokenization"
print(sequence)

# 第一步分词，也就是把文本变成Token
tokens = tokenizer.tokenize(sequence)
print(tokens) # 这里也可以看出来,BERT模型是子词分词

# 第二步token查表变成inputIDs
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

# 第三步还要给ids进行处理，加上模型需要的特殊字符
final_inputs = tokenizer.prepare_for_model(ids)
print(final_inputs)

# 解码过程decode
# 不仅将索引转回tokens，还会将相同单词的tokens组合在一起生成可读的句子
decoded_string = tokenizer.decode(ids)
print(decoded_string)


# 经过这三步生成的就跟直接调用tokenizer一样了
inputsid = tokenizer(sequence, padding=True, truncation=True, return_tensors="pt")
print(inputsid)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Using a Transformers network is simple, I like tokenization
['Using', 'a', 'Transformers', 'network', 'is', 'simple', ',', 'I', 'like', 'token', '##ization']
[7993, 170, 25267, 2443, 1110, 3014, 117, 146, 1176, 22559, 2734]
{'input_ids': [101, 7993, 170, 25267, 2443, 1110, 3014, 117, 146, 1176, 22559, 2734, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Using a Transformers network is simple, I like tokenization
{'input_ids': tensor([[  101,  7993,   170, 25267,  2443,  1110,  3014,   117,   146,  1176,
         22559,  2734,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [46]:
# 试试看！
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = [
    "I’ve been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]
print(inputs)

tokens = tokenizer.tokenize(inputs)
print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

['I’ve been waiting for a HuggingFace course my whole life.', 'I hate this so much!']
['i', '’', 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', 'i', 'hate', 'this', 'so', 'much', '!']
[1045, 1521, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 1045, 5223, 2023, 2061, 2172, 999]


In [27]:
# 模型需要一批输入
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
# 转换出来的向量是一维的
input_ids = torch.tensor(ids)
print(input_ids)

# 我们直接用tokenizer看看，到底喂给模型的是什么
tokenized_inputs = tokenizer(sequence, padding=True, truncation=True, return_tensors="pt")
# 看到了吗，他是两维的
print(tokenized_inputs["input_ids"])

final_inputs = torch.tensor([ids])
print(final_inputs)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])
tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])
tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])


In [28]:
# 会报错，只输入了一个句子但是模型需要一个句子列表
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# 增加一个维度，数据就输入成功了，复习一下怎么把他转变回来吧！
out = model(final_inputs)
print(out)

predictions = torch.nn.functional.softmax(out.logits, dim=-1)
print(predictions)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0037, 0.9963]], grad_fn=<SoftmaxBackward0>)
